## Spam Email Classifier with KNN using TF-IDF scores

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   The report file must be a well documented jupyter notebook, explaining the experiments you have performed, evaluation metrics and corresponding code. The code must run and be able to reproduce the accuracies, figures/graphs etc.
5.   For all the questions, you must create a train-validation data split and test the hyperparameter tuning on the validation set. Your jupyter notebook must reflect the same.
6.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

**Task: Given an email, classify it as spam or ham**

Given input text file ("emails.txt") containing 5572 email messages, with each row having its corresponding label (spam/ham) attached to it.

This task also requires basic pre-processing of text (like removing stopwords, stemming/lemmatizing, replacing email_address with 'email-tag', etc..).

You are required to find the tf-idf scores for the given data and use them to perform KNN using Cosine Similarity.

# CODE

### Import necessary libraries

In [44]:
import os
import re
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk.corpus
import nltk
nltk.download('punkt')
nltk.download('stopwords')
lemmer=nltk.WordNetLemmatizer()
stop_words=nltk.corpus.stopwords.words('english')
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /home/archit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/archit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Load dataset

In [40]:
path = "./emails.txt"
data = pd.read_csv(path, sep='\t', header=None)
data.rename({0: 'label', 1:'mail_text'}, axis='columns', inplace=True)

### Preprocess data

In [33]:
def del_up(text):
    '''
        convert all uppercase letters to lowercase
        replace email address with 'email-tag'
        stop words removal
        lemming the words
    '''
    text = text.lower()
    if text in stop_words:
        return False, text
    regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    if(re.fullmatch(regex, text)):
        return True, 'email-tag'
    return True, lemmer.lemmatize(text)


In [34]:
def tok_pun(sentence):
    token_list=[]
    for i in tokenizer.tokenize(sentence):
        a,b=del_up(i)
        if a==True:
            token_list.append(b)
    return token_list

In [41]:
data['tokenized_text'] = data['mail_text'].apply(lambda x: tok_pun(x))

In [42]:
data.head()

,label,mail_text,tokenized_text
0,ham,"Go until jurong point, crazy.. Available only ...","[go, jurong, point, crazy, available, bugis, n..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,U dun say so early hor... U c already then say...,"[u, dun, say, early, hor, u, c, already, say]"
4,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, think, go, usf, life, around, though]"


### Split data

In [45]:
train_X, test_X, train_y, test_y = train_test_split(data['tokenized_text'], data['label'], test_size=0.2, random_state=0)

### Train your KNN model (reuse previously iplemented model built from scratch) and test on your data

***1. Experiment with different distance measures [Euclidean distance, Manhattan distance, Hamming Distance] and compare with the Cosine Similarity distance results.***

***2. Explain which distance measure works best and why? Explore the distance measures and weigh their pro and cons in different application settings.***

***3. Report Mean Squared Error(MSE), Mean-Absolute-Error(MAE), R-squared (R2) score in a tabular form***

***4. Choose different K values (k=1,3,5,7,11,17,23,28) and experiment. Plot a graph showing R2 score vs k.***

### Train and test Sklearn's KNN classifier model on your data (use metric which gave best results on your experimentation with built-from-scratch model.)

***Compare both the models result.***

***What is the time complexity of training using KNN classifier?***

***What is the time complexity while testing? Is KNN a linear classifier or can it learn any boundary?***